# Importações e Instalações de Biblioteca



In [1]:
!pip install 'aif360[Reductions,inFairness]'
!pip install xlrd


Looking in indexes: https://pypi.org/simple, https://packagecloud.io/github/git-lfs/pypi/simple
Looking in indexes: https://pypi.org/simple, https://packagecloud.io/github/git-lfs/pypi/simple


In [2]:
from BibliotecasEFuncoes import * 
from PredicaoParaEducation import * 

2024-06-25 22:54:21.833034: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-25 22:54:25.322223: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/home/tuao/anaconda3/envs/aif360/lib/python3.11/site-packages/torch/_functorch/deprecated.py:61: UserWarning: We've integrated functorch into PyTorch. As the final step of the integration, functorch.vmap is deprecated as of PyTorch 2.0 and will be deleted in a future version of PyTorch >= 2.3. Please use torch.vmap instead; see the PyTorch 2.0 release notes and/or the torch.func migration guide for more details https://pytorch.org/docs/master/func.migrating.html
  warn_deprecated('vmap', 'torch.vmap')


# Lendo e processando a base de dados



Descrição Dos Dados no link: https://github.com/shrikant-temburwar/Loan-Prediction-Dataset

In [3]:

base = 'GitHubLoanContest'

GitHubLoan = pd.read_csv('Datasets/GitHubLoan-Prediction-Dataset.csv')
GitHubLoan.head()

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,LP001002,Male,No,0,Graduate,No,5849,0.0,NaN,360.0,1.0,Urban,Y
1,LP001003,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N
2,LP001005,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y
3,LP001006,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,Y
4,LP001008,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,Y


In [4]:
GitHubLoan.rename(columns={'Loan_Status' : 'class'}, inplace=True)
#separando rótulo (variável dependente y)
df_y = GitHubLoan['class']

## Codificando categorias

In [5]:
categorical_features = [i for i in GitHubLoan.columns if GitHubLoan.dtypes[i] in ['object', 'category']]
print(categorical_features)

['Loan_ID', 'Gender', 'Married', 'Dependents', 'Education', 'Self_Employed', 'Property_Area', 'class']


In [6]:
categorical_names = {}
encoders = {}

for feature in categorical_features:
  le = LabelEncoder()
  le.fit(GitHubLoan[feature])

  GitHubLoan[feature] = le.transform(GitHubLoan[feature])

  categorical_names[feature] = le.classes_
  encoders[feature] = le

In [7]:
GitHubLoan = GitHubLoan.dropna()
GitHubLoan = GitHubLoan.astype(int)

# todos devem estar codificados em int64
ghl = GitHubLoan
ghl.dtypes

Loan_ID              int64
Gender               int64
Married              int64
Dependents           int64
Education            int64
Self_Employed        int64
ApplicantIncome      int64
CoapplicantIncome    int64
LoanAmount           int64
Loan_Amount_Term     int64
Credit_History       int64
Property_Area        int64
class                int64
dtype: object

In [8]:
# Definições sobre os dados para avaliação de justiça
ghl.at[1, 'class'] = 0
label_name ='class'
protected_attribute_names = ['Gender', 'Education']
favorable_classes = np.where(ghl['class'] == 1)[0]

In [9]:
# Definindo as populações privilegiadas e não-privilegiadas
# Gênero
## Recuperando o indice
privileged_sex   = np.where(categorical_names['Gender'] == 'Male')[0]
unprivileged_sex = np.where(categorical_names['Gender'] == 'Female')[0]
## Estabelecendo formato, seguindo o formato da documentacao das classes ClassificationMetric and BinaryLabelDatasetMetric
privileged_groups   = [{'Gender' : privileged_sex}]
unprivileged_groups = [{'Gender' : unprivileged_sex}]

#Escolaridade
## Recuperando o indice
privileged_faixa_etaria   = np.where(categorical_names['Education'] == 'Graduate')[0]
unprivileged_faixa_etaria = np.where(categorical_names['Education'] == 'Not Graduate')[0]
## Estabelecendo formato, seguindo o formato da documentacao das classes ClassificationMetric and BinaryLabelDatasetMetric
privileged_groups_fe   = [{'Education' : privileged_faixa_etaria}]
unprivileged_groups_fe = [{'Education' : unprivileged_faixa_etaria}]


# Dados privilegiados
privileged_classes = [privileged_sex, privileged_faixa_etaria]

# Preparando experimentos

In [10]:
list_gbt_params, list_rf_params = preparando_experimentos()
print(list_gbt_params)
print(ghl['class'].value_counts())

Total de 36 conjuntos de parâmetros.
Total de 72 conjuntos de parâmetros.
[{'n_estimators': 500, 'min_samples_split': 300, 'learning_rate': 0.1, 'max_depth': 2, 'random_state': 2024}, {'n_estimators': 500, 'min_samples_split': 300, 'learning_rate': 0.1, 'max_depth': 3, 'random_state': 2024}, {'n_estimators': 500, 'min_samples_split': 300, 'learning_rate': 0.1, 'max_depth': 4, 'random_state': 2024}, {'n_estimators': 500, 'min_samples_split': 300, 'learning_rate': 0.2, 'max_depth': 2, 'random_state': 2024}, {'n_estimators': 500, 'min_samples_split': 300, 'learning_rate': 0.2, 'max_depth': 3, 'random_state': 2024}, {'n_estimators': 500, 'min_samples_split': 300, 'learning_rate': 0.2, 'max_depth': 4, 'random_state': 2024}, {'n_estimators': 500, 'min_samples_split': 300, 'learning_rate': 0.3, 'max_depth': 2, 'random_state': 2024}, {'n_estimators': 500, 'min_samples_split': 300, 'learning_rate': 0.3, 'max_depth': 3, 'random_state': 2024}, {'n_estimators': 500, 'min_samples_split': 300, 'lear

# Toda a base

## GBT

In [11]:
exp1_all_gbt = justica_exps_education(base, 'toda-a-base', ghl, list_gbt_params, 'gbt', categorical_features, 'class', favorable_classes, protected_attribute_names, privileged_classes, privileged_groups, unprivileged_groups, unprivileged_groups_fe, privileged_groups_fe, desbalanceamento=False)

<class 'dict'>
{'n_estimators': 500, 'min_samples_split': 300, 'learning_rate': 0.1, 'max_depth': 2, 'random_state': 2024}


0.82 de acuracia com desvio padrão de 0.05
0.74 de f1 com desvio padrão de 0.09
0.83 de precisão com desvio padrão de 0.05
0.76 de score no teste
STATISTICAL PARITY DIFFERENCE
Atributo sexo: 0.10606060606060608
Atributo educacao: -0.053980370774263875

DISPARATE IMPACT
Atributo sexo: 1.1414141414141414
Atributo educacao: 0.9357142857142857
 
AVERAGE ODDS DIFFERENCE
Atributo sexo: 0.09002525252525251
Atributo educacao: -0.03554439746300214
 
EQUAL OPPORTUNITY DIFFERENCE
Atributo sexo: 0.09924242424242424
Atributo educacao: 0.05681818181818177
{'index': 0, 'base': 'GitHubLoanContest', 'attr_set': 'toda-a-base', 'modelo': 'gbt', 'param_set': "{'n_estimators': 500, 'min_samples_split': 300, 'learning_rate': 0.1, 'max_depth': 2, 'random_state': 2024}", '10fold-acuracia': 0.818918918918919, '10fold-f1': 0.7389036104489998, '10fold-precisao': 0.8256138962590575, 'test-score': 0.7610062893081762, 'spd_sexo': 0.10606060606060608, 'spd_educacao': -0.053980370774263875, 'di_sexo': 1.1414141414141

In [12]:
csv_result = pd.DataFrame(exp1_all_gbt)
csv_result.to_csv('Resultados/04-GitHubLoanContest_GBT.csv')
print(csv_result)

    index               base     attr_set modelo                                          param_set  10fold-acuracia  10fold-f1  10fold-precisao  test-score  spd_sexo  spd_educacao   di_sexo  di_educacao  aod_sexo  aod_educacao  eod_sexo  eod_educacao
0       0  GitHubLoanContest  toda-a-base    gbt  {'n_estimators': 500, 'min_samples_split': 300...         0.818919   0.738904         0.825614    0.761006  0.106061     -0.053980  1.141414     0.935714  0.090025     -0.035544  0.099242      0.056818
1       1  GitHubLoanContest  toda-a-base    gbt  {'n_estimators': 500, 'min_samples_split': 300...         0.783784   0.700646         0.799982    0.773585 -0.051908      0.001969  0.935115     1.002632 -0.107499      0.038114 -0.062366      0.023596
2       2  GitHubLoanContest  toda-a-base    gbt  {'n_estimators': 500, 'min_samples_split': 300...         0.783784   0.708980         0.806415    0.767296  0.034738     -0.096941  1.049841     0.869703  0.015568     -0.049042  0.011905      0

## RF


In [11]:
exp2_all_rf = justica_exps_education(base, 'toda-a-base', ghl, list_rf_params, 'rf', categorical_features, 'class', favorable_classes, protected_attribute_names, privileged_classes, unprivileged_groups, privileged_groups, unprivileged_groups_fe, privileged_groups_fe, desbalanceamento=False)

<class 'dict'>
{'n_estimators': 500, 'min_samples_split': 300, 'criterion': 'gini', 'max_depth': 2, 'random_state': 2024}
0.71 de acuracia com desvio padrão de 0.01
0.41 de f1 com desvio padrão de 0.00
0.71 de precisão com desvio padrão de 0.01
0.65 de score no teste
STATISTICAL PARITY DIFFERENCE
Atributo sexo: 0.0
Atributo educacao: 0.0

DISPARATE IMPACT
Atributo sexo: 1.0
Atributo educacao: 1.0
 
AVERAGE ODDS DIFFERENCE
Atributo sexo: 0.0
Atributo educacao: 0.0
 
EQUAL OPPORTUNITY DIFFERENCE
Atributo sexo: 0.0
Atributo educacao: 0.0
{'index': 0, 'base': 'GitHubLoanContest', 'attr_set': 'toda-a-base', 'modelo': 'rf', 'param_set': "{'n_estimators': 500, 'min_samples_split': 300, 'criterion': 'gini', 'max_depth': 2, 'random_state': 2024}", '10fold-acuracia': 0.7081081081081082, '10fold-f1': 0.41453373015873013, '10fold-precisao': 0.7081081081081082, 'test-score': 0.6540880503144654, 'spd_sexo': 0.0, 'spd_educacao': 0.0, 'di_sexo': 1.0, 'di_educacao': 1.0, 'aod_sexo': 0.0, 'aod_educacao'

KeyboardInterrupt: 

In [12]:
csv_result2 = pd.DataFrame(exp2_all_rf)
csv_result2.to_csv('Resultados/09-GitHubLoanContest_RF.csv')
print(csv_result2)

    index               base     attr_set modelo                                          param_set  10fold-acuracia  10fold-f1  10fold-precisao  test-score  spd_sexo  spd_faixa_etaria  di_sexo  di_faixa_etaria  aod_sexo  aod_faixa_etaria  eod_sexo  eod_faixa_etaria
0       0  GitHubLoanContest  toda-a-base     rf  {'n_estimators': 500, 'min_samples_split': 300...         0.708108   0.414534         0.708108    0.654088       0.0               0.0      1.0              1.0       0.0               0.0       0.0               0.0
1       1  GitHubLoanContest  toda-a-base     rf  {'n_estimators': 500, 'min_samples_split': 300...         0.694595   0.409857         0.694595    0.685535       0.0               0.0      1.0              1.0       0.0               0.0       0.0               0.0
2       2  GitHubLoanContest  toda-a-base     rf  {'n_estimators': 500, 'min_samples_split': 300...         0.697297   0.410804         0.697297    0.679245       0.0               0.0      1.0      